# Column Generation

Imports

In [5]:
import sys
import math
from cplex import Cplex
import cplex
from cplex.exceptions import CplexError
import time

## Problem Definition

In [6]:
# Assuming we have necessary imports for data handling
# For the database connection, we'll simulate data loading

class ColumnGen:
    def __init__(self, instance, cvrplib:bool=True):
        self.instance = instance
        self.cvrplib = cvrplib # Instance file format
        self.depot_start = None
        self.depot_end = None
        self.capacity = None
        self.all_customers = {}
        self.all_nodes = {}
        self.paths = []
        # self.distance_matrix = None
        self.masterproblem = None
        self.subproblem = None
        self.logger = Logger()
        self.ReadData()
        self.masterproblem = self.MasterProblem(self)
        self.subproblem = self.SubProblem(self)

    class Node:
        def __init__(self, external_id, x, y, parent):
            self.parent = parent  # Reference to ColumnGen instance
            self.id = len(self.parent.all_nodes)
            self.id_external = external_id
            self.xcoord = x
            self.ycoord = y
            self.parent.all_nodes[self.id] = self

        def dist_to_node(self, node_to):
            # return self.parent.distance_matrix[self.id_external][node_to.id_external] 
            return math.hypot(self.xcoord - node_to.xcoord, self.ycoord - node_to.ycoord)
    
    class Depot(Node):
        def __init__(self, external_id, x, y, parent):
            super().__init__(external_id, x, y, parent)

    class Customer(Node):
        def __init__(self, external_id, x, y, demand, parent):
            super().__init__(external_id, x, y, parent)
            self.demand = demand
            self.parent.all_customers[self.id] = self

        def d(self):
            return self.demand

    class Path:
        def __init__(self, stops_new_path, parent):
            self.parent = parent  # Reference to ColumnGen instance
            self.customers = []
            # From first customer in path to last customer prior to depot
            for i in range(1, len(stops_new_path) - 1):
                self.customers.append(self.parent.all_customers[stops_new_path[i]])
            self.cost = self.calculateCost()
            self.id = len(self.parent.paths)
            self.y = None  # Variable in master problem
            self.parent.paths.append(self)

        def calculateCost(self):
            if len(self.customers) > 0:
                cost = self.parent.depot_start.dist_to_node(self.customers[0])
                for i in range(1, len(self.customers)):
                    cost += self.customers[i - 1].dist_to_node(self.customers[i])
                cost += self.customers[-1].dist_to_node(self.parent.depot_end)
            else:
                cost = 0
            return cost

        def a(self, customer):
            return 1 if customer in self.customers else 0

        def displayInfo(self):
            print(f"Path id : {self.id}")
            stops = '->'.join([str(c.id_external) for c in self.customers])
            print(f"Stops : depot->{stops}->depot")
            print(f"Cost : {self.cost}")

    class MasterProblem:
        def __init__(self, parent):
            self.parent = parent  # Reference to ColumnGen instance
            self.cplex = None
            self.total_cost = None
            self.row_customers = {}
            self.pi = {}
            self.mipConversion = []
            self.lastObjValue = None
            self.createModel()
            self.createDefaultPaths()
            # Parameters.configureCplex(self)  # Configure CPLEX parameters if needed

        def createDefaultPaths(self):
            for c in self.parent.all_customers.values():
                new_path = [self.parent.depot_start.id, c.id, self.parent.depot_end.id]
                self.addNewColumn(self.parent.Path(new_path, self.parent))
                self.cplex.write('master_step0.lp')

        def createModel(self): 
            try:
                self.cplex = Cplex()
                self.cplex.objective.set_sense(self.cplex.objective.sense.minimize)
                self.total_cost = []
                # Add constraints for each customer
                for customer in self.parent.all_customers.values():
                    name = f"cust_{customer.id}"
                    # constraint = [[[]], [[]]] #FIXME?
                    # Initially, no variables are associated with the constraint
                    # We'll add coefficients when adding columns (paths)
                    self.cplex.linear_constraints.add(
                        lin_expr=[cplex.SparsePair([], [])],
                        senses=["E"],
                        rhs=[1],
                        names=[name]
                    )
                    self.row_customers[customer] = name
                    # self.cplex.linear_constraints.add(lin_expr=constraint, senses="E", rhs=[1], names=[name])
            except CplexError as e:
                print("Cplex exception caught:", e)

        def addNewColumn(self, path):
            try:
                num_paths = len(self.parent.paths)
                var_name = f"y_{path.id}"
                path.y = var_name
                self.cplex.variables.add(obj=[path.cost], lb=[0], ub=[1], types=["C"], names=[var_name])
                # Add coefficients to customer constraints
                for customer in self.parent.all_customers.values():
                    coef = path.a(customer)
                    if coef != 0:
                        row_index = list(self.cplex.linear_constraints.get_names()).index(self.row_customers[customer])
                        col_index = self.cplex.variables.get_names().index(var_name)
                        self.cplex.linear_constraints.set_coefficients(row_index, col_index, coef)
            except CplexError as e:
                print("Cplex exception caught:", e)

        def solveRelaxation(self):
            print(f'\n\n ######## Solving Relaxation: MP ######## \n\n')
            try:
                self.cplex.set_problem_type(self.cplex.problem_type.LP)
                self.cplex.write('master.lp')
                self.cplex.solve()
                self.saveDualValues()
                self.lastObjValue = self.cplex.solution.get_objective_value()
            except CplexError as e:
                print("Cplex exception caught:", e)

        def saveDualValues(self):
            try:
                duals = self.cplex.solution.get_dual_values()
                constraint_names = self.cplex.linear_constraints.get_names()
                for customer in self.parent.all_customers.values():
                    idx = constraint_names.index(self.row_customers[customer])
                    self.pi[customer] = duals[idx]
            except CplexError as e:
                print("Cplex exception caught:", e)

        def convertToMIP(self):
            try:
                var_names = [path.y for path in self.parent.paths]
                indices = [self.cplex.variables.get_names().index(name) for name in var_names]
                self.cplex.variables.set_types(zip(indices, ['B'] * len(indices)))
            except CplexError as e:
                print("Cplex exception caught:", e)

        def solveMIP(self):
            try:
                self.convertToMIP()
                self.cplex.set_problem_type(self.cplex.problem_type.MILP)
                self.cplex.solve()
                if self.cplex.solution.get_status() == self.cplex.solution.status.optimal:
                    self.displaySolution()
                    # self.logger.writeLog(self.parent.instance, self.cplex.solution.get_objective_value(), self.cplex.solution.MIP.get_best_objective())
                else:
                    print("Integer solution not found")
            except CplexError as e:
                print("Cplex exception caught:", e)

        def displaySolution(self):
            try:
                for path in self.parent.paths:
                    idx = self.cplex.variables.get_names().index(path.y)
                    value = self.cplex.solution.get_values(idx)
                    if value > 0.99999:
                        path.displayInfo()
            except CplexError as e:
                print("Cplex exception caught:", e)

    class SubProblem:
        def __init__(self, parent):
            self.parent = parent  # Reference to ColumnGen instance
            self.cplex = None
            self.x = {}
            self.u = {}  # Continuous variables for cumulative load at customers
            self.x_var_indices = {}
            self.reduced_cost = None
            self.constraints = []
            self.lastObjValue = None
            self.lastObjValueRelaxed = None
            self.createModel()
            self.cplex.parameters.mip.pool.capacity.set(2)  # Set the maximum number of solutions
            self.cplex.parameters.mip.pool.intensity.set(4)  # Higher value for more diverse solutions

            # self.setPriority()  # Set variable priorities if needed
            # Parameters.configureCplex(self)
            # self.mip_call_back = self.MyMipCallBack()

        def createModel(self):
            try:
                self.cplex = Cplex()
                self.cplex.objective.set_sense(self.cplex.objective.sense.minimize)

                # Variables
                nodes = self.parent.all_nodes
                num_nodes = len(nodes)
                all_customers = self.parent.all_customers

                Q = self.parent.capacity  # Vehicle capacity
                q = {i: all_customers[i].d() for i in all_customers}  # Customer demands

                self.x = {}
                x_var_names = []
                for i in nodes:
                    for j in nodes:
                        if i != j:
                            var_name = f"x_{i}_{j}"
                            self.x[(i, j)] = var_name
                            x_var_names.append(var_name)
                self.cplex.variables.add(names=list(self.x.values()), types=["B"] * len(self.x))

                # Store variable indices for quick access
                x_var_indices = self.cplex.variables.get_indices(x_var_names)
                self.x_var_indices = {var_name: idx for var_name, idx in zip(x_var_names, x_var_indices)}
                
                # Variables: u[i] - continuous variables representing cumulative load at customer i
                # Defined for customers only (excluding depot)
                for i in all_customers:
                    var_name = f"u_{i}"
                    self.u[i] = var_name
                self.cplex.variables.add(
                    names=list(self.u.values()),
                    lb=[q[i] for i in self.u],  # Lower bound: demand at customer i
                    ub=[Q] * len(self.u),       # Upper bound: vehicle capacity
                    types=["C"] * len(self.u)
                )

                # Constraints

                # 1. Capacity Constraint
                # sum over i in customers, sum over j, demand[i] * x[i,j] <= vehicle capacity
                capacity_expr = []
                capacity_coef = []
                for i in all_customers:
                    for j in nodes:
                        if (i, j) in self.x:
                            capacity_expr.append(self.x[(i, j)])
                            capacity_coef.append(all_customers[i].d())
                self.cplex.linear_constraints.add(
                    lin_expr=[cplex.SparsePair(capacity_expr, capacity_coef)],
                    senses=["L"],
                    rhs=[self.parent.capacity],
                    names=["Capacity"]
                )

                # 2. Start from Depot Constraint
                depot_start_id = self.parent.depot_start.id
                start_expr = []
                for j in nodes:
                    if depot_start_id != j and (depot_start_id, j) in self.x:
                        start_expr.append(self.x[(depot_start_id, j)])
                self.cplex.linear_constraints.add(
                    lin_expr=[cplex.SparsePair(start_expr, [1]*len(start_expr))],
                    senses=["E"],
                    rhs=[1],
                    names=["StartDepot"]
                )

                # 3. Flow Conservation Constraints
                # For each customer h, sum over i x[i,h] - sum over j x[h,j] == 0
                for h in all_customers:
                    flow_in = []
                    flow_out = []
                    for i in nodes:
                        if (i, h) in self.x:
                            flow_in.append(self.x[(i, h)])
                    for j in nodes:
                        if (h, j) in self.x:
                            flow_out.append(self.x[(h, j)])
                    expr_vars = flow_in + flow_out
                    expr_coefs = [1]*len(flow_in) + [-1]*len(flow_out)
                    self.cplex.linear_constraints.add(
                        lin_expr=[cplex.SparsePair(expr_vars, expr_coefs)],
                        senses=["E"],
                        rhs=[0],
                        names=[f"Flow_{h}"]
                    )

                # 4. End at Depot Constraint
                depot_end_id = self.parent.depot_end.id
                end_expr = []
                for i in nodes:
                    if i != depot_end_id and (i, depot_end_id) in self.x:
                        end_expr.append(self.x[(i, depot_end_id)])
                self.cplex.linear_constraints.add(
                    lin_expr=[cplex.SparsePair(end_expr, [1]*len(end_expr))],
                    senses=["E"],
                    rhs=[1],
                    names=["EndDepot"]
                )

                # 5. Subtour Elimination Constraints (MTZ Constraints)
                # u[i] - u[j] + Q * x[i,j] <= Q - q[j], for all i != j, i, j != depot
                for i in all_customers:
                    for j in all_customers:
                        if i != j and (i, j) in self.x:
                            expr_vars = [self.u[i], self.u[j], self.x[(i, j)]]
                            expr_coefs = [1, -1, Q]
                            rhs = Q - q[j]
                            self.cplex.linear_constraints.add(
                                lin_expr=[cplex.SparsePair(expr_vars, expr_coefs)],
                                senses=["L"],
                                rhs=[rhs],
                                names=[f"SubtourEl_{i}_{j}"]
                            )

                # Additional constraints to link u[i] with the depot arcs
                # For arcs from depot to customers
                for j in all_customers:
                    if (depot_start_id, j) in self.x:
                        # u[j] >= q[j] * x[depot, j]
                        expr_vars = [self.u[j], self.x[(depot_start_id, j)]]
                        expr_coefs = [1, -q[j]]
                        self.cplex.linear_constraints.add(
                            lin_expr=[cplex.SparsePair(expr_vars, expr_coefs)],
                            senses=["G"],
                            rhs=[0],
                            names=[f"LoadFromDepot_{j}"]
                        )
                
                # 6. Prohibited Moves Constraints 
                prohibited_expr = []
                for i in nodes:
                    # No arcs from depot_end to any node
                    if (depot_end_id, i) in self.x:
                        prohibited_expr.append(self.x[(depot_end_id, i)])
                    # No arcs to depot_start from any node
                    if (i, depot_start_id) in self.x:
                        prohibited_expr.append(self.x[(i, depot_start_id)])
                    # No loops (x[i,i] = 0)
                    if (i, i) in self.x:
                        prohibited_expr.append(self.x[(i, i)])
                self.cplex.linear_constraints.add(
                    lin_expr=[cplex.SparsePair(prohibited_expr, [1]*len(prohibited_expr))],
                    senses=["E"],
                    rhs=[0],
                    names=["ProhibitedMoves"]
                )

                # Initialize the objective function (will be updated later)
                # self.cplex.objective.set_linear([(self.x[arc], 0) for arc in self.x])
                
                # Tweak some CPLEX parameters so that CPLEX has a harder time to
                # solve the model and our cut separators can actually kick in.
                self.cplex.parameters.mip.strategy.heuristicfreq.set(-1)
                self.cplex.parameters.mip.cuts.mircut.set(-1)
                self.cplex.parameters.mip.cuts.implied.set(-1)
                self.cplex.parameters.mip.cuts.gomory.set(-1)
                self.cplex.parameters.mip.cuts.flowcovers.set(-1)
                self.cplex.parameters.mip.cuts.pathcut.set(-1)
                self.cplex.parameters.mip.cuts.liftproj.set(-1)
                self.cplex.parameters.mip.cuts.zerohalfcut.set(-1)
                self.cplex.parameters.mip.cuts.cliques.set(-1)
                self.cplex.parameters.mip.cuts.covers.set(-1)
                self.cplex.parameters.threads.set(1)
                self.cplex.parameters.clocktype.set(1)
                self.cplex.parameters.timelimit.set(1800)

            except CplexError as e:
                print("Cplex exception caught:", e)

        def updateReducedCost(self):
            try:
                # Objective function
                obj = []
                obj_names = []
                for (i, j) in self.x:
                    t_ij = self.parent.all_nodes[i].dist_to_node(self.parent.all_nodes[j])
                    if i in self.parent.all_customers:
                        pi_i = self.parent.masterproblem.pi[self.parent.all_customers[i]]
                        cost = t_ij - pi_i
                    else:
                        cost = t_ij
                    obj.append(cost)
                    obj_names.append(self.x[(i, j)])
                self.cplex.objective.set_linear(zip(obj_names, obj))
                self.cplex.objective.set_sense(self.cplex.objective.sense.minimize)
                # Constraints can be added similarly
            except CplexError as e:
                print("Cplex exception caught:", e)

        def solve(self):
            print(f'\n\n ######## Solving Subproblem ######## \n\n')
            try:
                # Solve the subproblem
                self.cplex.write('sub.lp')
                self.cplex.solve()
                self.lastObjValue = self.cplex.solution.get_objective_value()
                self.lastObjValueRelaxed = self.cplex.solution.MIP.get_best_objective()
                nPool = self.cplex.solution.pool.get_num()
                for i in range(nPool):
                    objval = self.cplex.solution.pool.get_objective_value(i)
                    if objval < -1e-6:
                        print(f'\n\n ######## Saving Path: {i} | Pool Size: {nPool} ######## \n\n')
                        self.savePath(i)  # Pass the solution index
            except CplexError as e:
                print("Cplex exception caught:", e)

        # def savePath(self, nSol):
        #     try:
        #         depot_start_id = self.parent.depot_start.id
        #         depot_end_id = self.parent.depot_end.id

        #         stops_new_path = []
        #         for i in self.parent.all_customers:
        #             if self.cplex.solution.pool.get_values(nSol, self.x[(depot_start_id, i)]) > 0.99999:
        #                 stops_new_path.extend([depot_start_id,i])
        #                 current = i
        #                 while current != depot_end_id:
        #                     for j in self.parent.all_nodes:
        #                         if self.cplex.solution.pool.get_values(nSol, self.x[(i, j)]) > 0.99999:
        #                             stops_new_path.append(j)
        #                             current = j
        #                             break
        #                 break # Path found, no need to check other customers
                
        #     except CplexError as e:
        #         pass
        
        def savePath(self, nSol):
            try:
                # Extract variable names and indices
                x_arcs = list(self.x.keys())
                x_var_names = [self.x[arc] for arc in x_arcs]
                x_var_indices = [self.cplex.variables.get_indices(var_name) for var_name in x_var_names]

                # Extract solution values for the specific solution index
                x_values = self.cplex.solution.pool.get_values(nSol, x_var_indices)

                # Map arcs to their solution values
                x_dict = {arc: x_values[i] for i, arc in enumerate(x_arcs)}

                # Reconstruct the path from x_dict
                depot_start_id = self.parent.depot_start.id
                depot_end_id = self.parent.depot_end.id
                nodes = self.parent.all_nodes
                all_customers = self.parent.all_customers
                stops_new_path = []

                # Find the customer directly connected from the depot
                for i in all_customers:
                    if x_dict.get((depot_start_id, i), 0) > 0.99999:
                        stops_new_path = [depot_start_id, i]
                        current_node = i
                        # print(len(nodes), nodes , depot_end_id, depot_start_id)
                        
                        leave_flag = False
                        while current_node != depot_end_id:
                            for j in nodes:
                                if (j == 0) or (j == current_node):
                                    continue
                                elif x_dict.get((current_node, j), 0) > 0.99999:
                                    stops_new_path.append(j)
                                    current_node = j
                                    print(f"Current path is: {stops_new_path}")
                                    break
                                elif j == len(nodes)-1:
                                    print(f"No arc found in {stops_new_path} from node {current_node}")
                                    leave_flag = True
                            if leave_flag: break
                        break  # Path found, no need to check other customers
                print(f"###### Path: {stops_new_path} #######")
                if stops_new_path and stops_new_path[-1] == depot_end_id:
                    # Valid path found
                    new_path = self.parent.Path(stops_new_path, self.parent)
                    self.parent.masterproblem.addNewColumn(new_path)
                    print(f"Added new path from solution {nSol}: {stops_new_path}")
                else:
                    print(f"No valid path ends at depot in solution {nSol}.")
            except CplexError as e:
                print("Cplex exception caught in savePath():", e)


    def ReadData(self):

        # Method for reading standard cvrplib instances
        def read_cvrplib_instance(file_path):
            """
            Reads a CVRP instance from a CVRPLIB file and returns the node coordinates, demands, and capacity.
            """
            customers = []
            depot_coords = tuple()
            capacity = 0
            with open(file_path, 'r') as file:
                section = None
                for line in file:
                    line = line.strip()
                    if line.startswith('CAPACITY'):
                        capacity = int(line.split(':')[1].strip())
                    elif line.startswith('NODE_COORD_SECTION'):
                        section = 'NODE_COORD_SECTION'
                    elif line.startswith('DEMAND_SECTION'):
                        section = 'DEMAND_SECTION'
                    elif line.startswith('DEPOT_SECTION'):
                        break
                    elif section == 'NODE_COORD_SECTION' and line != '':
                        parts = line.split()

                        node_id = int(parts[0])
                        x_coord = int(parts[1])
                        y_coord = int(parts[2])

                        if node_id == 0:
                            depot_coords = (x_coord, y_coord)
                        else:
                            customers.append({
                                'number': node_id,
                                'x': x_coord,
                                'y': y_coord,
                                'demand': 0
                            })
                            # print(customers)
                    elif section == 'DEMAND_SECTION' and line != '':
                        parts = line.split()

                        node_id = int(parts[0])
                        demand = float(parts[1])

                        if node_id != 0:
                            customers[node_id-1]['demand'] = demand
            return capacity, depot_coords, customers

        # Method for reading non cvrplib instances
        def read_instance(file_path):
            """
            Reads the CVRP instance from a file.

            The file format is assumed to be:
            - The first line contains two numbers: number of customers and best known solution (ignored).
            - The second line contains the vehicle capacity.
            - The third line contains the depot coordinates (x y).
            - The following lines contain customer data: customer number, x, y, demand.

            Returns:
            - capacity: Vehicle capacity.
            - depot_coords: Coordinates of the depot (tuple).
            - customers: List of customer dictionaries with keys 'number', 'x', 'y', 'demand'.
            """
            with open(file_path, 'r') as f:
                lines = f.readlines()

            # Remove any empty lines or lines with only whitespace
            lines = [line.strip() for line in lines if line.strip()]

            idx = 0
            num_customers, _ = map(float, lines[idx].split())
            num_customers = int(num_customers)
            idx += 1
            capacity = int(lines[idx])
            idx += 1
            depot_coords = tuple(map(int, lines[idx].split()))
            idx += 1

            customers = []
            for _ in range(num_customers):
                customer_info = lines[idx].split()
                idx += 1
                customer_number = int(customer_info[0])
                x = int(customer_info[1])
                y = int(customer_info[2])
                demand = int(customer_info[3])
                customers.append({
                    'number': customer_number,
                    'x': x,
                    'y': y,
                    'demand': demand
                })

            return capacity, depot_coords, customers

        filepath = f"../Instances/{self.instance}.txt"
        if self.cvrplib:
            capacity, depot_coords, customers = read_cvrplib_instance(filepath)
        else:
            capacity, depot_coords, customers = read_instance(filepath)

        self.capacity = capacity
        self.depot_start = self.Depot(
            0, depot_coords[0], depot_coords[1], self
        )
        self.depot_end = self.Depot(
            0, depot_coords[0], depot_coords[1], self
        )

        # Add customers
        for c in customers:
            external_id = c['number']
            x = c['x']
            y = c['y']
            demand = c['demand']
            self.Customer(external_id, x, y, demand, self)

    def runColumnGeneration(self):
        iteration_counter = 0
        while iteration_counter < 35:
            iteration_counter += 1
            print(f'\n\n ######## Iteration: {iteration_counter} ######## \n\n')
            self.masterproblem.solveRelaxation()
            self.subproblem.updateReducedCost()
            self.subproblem.solve()
            self.displayIteration(iteration_counter)
            if self.subproblem.lastObjValue >= -1e-6:
                break
        self.masterproblem.solveMIP()
        print('Final Solution: ', self.masterproblem.solution.get_values())

    def displayIteration(self, iter):
        if iter % 20 == 0 or iter == 1:
            print()
            print(f"{'Iteration':>9} {'Time':>9} {'nPaths':>9} {'MP lb':>12} {'SP lb':>12} {'SP int':>12}")
        print(f"{iter:9d} {time.time() / 60:9.1f} {len(self.paths):9d} {self.masterproblem.lastObjValue:12.4f} {self.subproblem.lastObjValueRelaxed:12.4f} {self.subproblem.lastObjValue:12.4f}")

class Logger:
    def timeStamp(self):
        return time.time()

# Assuming Parameters is a class or module containing configuration settings
# class Parameters:
#     capacity = 100.0
#     infinity = 1e20
#     ColGen = type('ColGen', (), {'subproblemObjVal': -1e-6, 'zero_reduced_cost': -1e-6, 'zero_reduced_cost_AbortColGen': -1e-6})


In [7]:
a = [0]
if a and a[-1] == 0: print('hi')

hi


## Testing

In [8]:
if __name__ == '__main__':
    instance_name = 'mini_test' # 'Adj-P-n19-k2'
    colgen = ColumnGen(instance_name, cvrplib=True)  
    colgen.runColumnGeneration()

CPLEX Error  1222: Duplicate entry or entries.


Cplex exception caught: CPLEX Error  1222: Duplicate entry or entries.


 ######## Iteration: 1 ######## 




 ######## Solving Relaxation: MP ######## 


Version identifier: 22.1.1.0 | 2022-11-28 | 9160aff4d
CPXPARAM_Read_DataCheck                          1
Tried aggregator 1 time.
LP Presolve eliminated 4 rows and 4 columns.
All rows and columns eliminated.
Presolve time = 0.00 sec. (0.00 ticks)




 ######## Solving Subproblem ######## 


Version identifier: 22.1.1.0 | 2022-11-28 | 9160aff4d
CPXPARAM_Read_DataCheck                          1
CPXPARAM_MIP_Pool_Capacity                       2
CPXPARAM_MIP_Pool_Intensity                      4
Found incumbent of value 0.000000 after 0.00 sec. (0.00 ticks)
Tried aggregator 1 time.
MIP Presolve eliminated 4 rows and 0 columns.
Reduced MIP has 19 rows, 34 columns, and 106 nonzeros.
Reduced MIP has 30 binaries, 0 generals, 0 SOSs, and 0 indicators.
Presolve time = 0.00 sec. (0.04 ticks)
Probing time = 0.00 sec. (0.01 ticks)
Tried aggregator 1 time.
Reduced MIP has 19 rows, 34 columns, and 106 nonzeros.
Reduced MIP has 30 binaries, 0 generals, 0 SOSs, and 0 indicators.
Presolve time = 0.00 sec. (0.04 ticks)
Probing time = 0.00 sec. (0.01 ticks)
Clique table members: 8.
MIP emphasis: balance optimality and feasibility.
MIP search method: dynamic search.
Parallel mode: deterministic, using up to 16 threads.
Root relaxation solution time



 ######## Solving Subproblem ######## 


Version identifier: 22.1.1.0 | 2022-11-28 | 9160aff4d
CPXPARAM_Read_DataCheck                          1
CPXPARAM_MIP_Pool_Capacity                       2
CPXPARAM_MIP_Pool_Intensity                      4
2 of 2 MIP starts provided solutions.
MIP start 'm1' defined initial solution with objective -100.3396.
Tried aggregator 1 time.
MIP Presolve eliminated 4 rows and 0 columns.
Reduced MIP has 19 rows, 34 columns, and 106 nonzeros.
Reduced MIP has 30 binaries, 0 generals, 0 SOSs, and 0 indicators.
Presolve time = 0.00 sec. (0.04 ticks)
Probing time = 0.00 sec. (0.01 ticks)
Tried aggregator 1 time.
Reduced MIP has 19 rows, 34 columns, and 106 nonzeros.
Reduced MIP has 30 binaries, 0 generals, 0 SOSs, and 0 indicators.
Presolve time = 0.00 sec. (0.04 ticks)
Probing time = 0.00 sec. (0.01 ticks)
Clique table members: 8.
MIP emphasis: balance optimality and feasibility.
MIP search method: dynamic search.
Parallel mode: deterministic, using up to 



 ######## Solving Subproblem ######## 


Version identifier: 22.1.1.0 | 2022-11-28 | 9160aff4d
CPXPARAM_Read_DataCheck                          1
CPXPARAM_MIP_Pool_Capacity                       2
CPXPARAM_MIP_Pool_Intensity                      4
2 of 2 MIP starts provided solutions.
MIP start 'm1' defined initial solution with objective -56.1619.
Tried aggregator 1 time.
MIP Presolve eliminated 4 rows and 0 columns.
Reduced MIP has 19 rows, 34 columns, and 106 nonzeros.
Reduced MIP has 30 binaries, 0 generals, 0 SOSs, and 0 indicators.
Presolve time = 0.00 sec. (0.04 ticks)
Probing time = 0.00 sec. (0.01 ticks)
Tried aggregator 1 time.
Reduced MIP has 19 rows, 34 columns, and 106 nonzeros.
Reduced MIP has 30 binaries, 0 generals, 0 SOSs, and 0 indicators.
Presolve time = 0.00 sec. (0.04 ticks)
Probing time = 0.00 sec. (0.01 ticks)
Clique table members: 8.
MIP emphasis: balance optimality and feasibility.
MIP search method: dynamic search.
Parallel mode: deterministic, using up to 1

...

Version identifier: 22.1.1.0 | 2022-11-28 | 9160aff4d
CPXPARAM_Read_DataCheck                          1
CPXPARAM_MIP_Pool_Capacity                       2
CPXPARAM_MIP_Pool_Intensity                      4
2 of 2 MIP starts provided solutions.
MIP start 'm1' defined initial solution with objective -52.1028.
Tried aggregator 1 time.
MIP Presolve eliminated 4 rows and 0 columns.
Reduced MIP has 19 rows, 34 columns, and 106 nonzeros.
Reduced MIP has 30 binaries, 0 generals, 0 SOSs, and 0 indicators.
Presolve time = 0.00 sec. (0.04 ticks)
Probing time = 0.00 sec. (0.01 ticks)
Tried aggregator 1 time.
Reduced MIP has 19 rows, 34 columns, and 106 nonzeros.
Reduced MIP has 30 binaries, 0 generals, 0 SOSs, and 0 indicators.
Presolve time = 0.00 sec. (0.04 ticks)
Probing time = 0.00 sec. (0.01 ticks)
Clique table members: 8.
MIP emphasis: balance optimality and feasibility.
MIP search method: dynamic search.
Parallel mode: deterministic, using up to 16 threads.
Root relaxation solution time = 



 ######## Solving Subproblem ######## 


Version identifier: 22.1.1.0 | 2022-11-28 | 9160aff4d
CPXPARAM_Read_DataCheck                          1
CPXPARAM_MIP_Pool_Capacity                       2
CPXPARAM_MIP_Pool_Intensity                      4
2 of 2 MIP starts provided solutions.
MIP start 'm1' defined initial solution with objective -52.1028.
Tried aggregator 1 time.
MIP Presolve eliminated 4 rows and 0 columns.
Reduced MIP has 19 rows, 34 columns, and 106 nonzeros.
Reduced MIP has 30 binaries, 0 generals, 0 SOSs, and 0 indicators.
Presolve time = 0.00 sec. (0.04 ticks)
Probing time = 0.00 sec. (0.01 ticks)
Tried aggregator 1 time.
Reduced MIP has 19 rows, 34 columns, and 106 nonzeros.
Reduced MIP has 30 binaries, 0 generals, 0 SOSs, and 0 indicators.
Presolve time = 0.00 sec. (0.04 ticks)
Probing time = 0.00 sec. (0.01 ticks)
Clique table members: 8.
MIP emphasis: balance optimality and feasibility.
MIP search method: dynamic search.
Parallel mode: deterministic, using up to 1



 ######## Solving Subproblem ######## 


Version identifier: 22.1.1.0 | 2022-11-28 | 9160aff4d
CPXPARAM_Read_DataCheck                          1
CPXPARAM_MIP_Pool_Capacity                       2
CPXPARAM_MIP_Pool_Intensity                      4
2 of 2 MIP starts provided solutions.
MIP start 'm1' defined initial solution with objective -52.1028.
Tried aggregator 1 time.
MIP Presolve eliminated 4 rows and 0 columns.
Reduced MIP has 19 rows, 34 columns, and 106 nonzeros.
Reduced MIP has 30 binaries, 0 generals, 0 SOSs, and 0 indicators.
Presolve time = 0.00 sec. (0.04 ticks)
Probing time = 0.00 sec. (0.01 ticks)
Tried aggregator 1 time.
Reduced MIP has 19 rows, 34 columns, and 106 nonzeros.
Reduced MIP has 30 binaries, 0 generals, 0 SOSs, and 0 indicators.
Presolve time = 0.00 sec. (0.04 ticks)
Probing time = 0.00 sec. (0.01 ticks)
Clique table members: 8.
MIP emphasis: balance optimality and feasibility.
MIP search method: dynamic search.
Parallel mode: deterministic, using up to 1



 ######## Solving Subproblem ######## 


Version identifier: 22.1.1.0 | 2022-11-28 | 9160aff4d
CPXPARAM_Read_DataCheck                          1
CPXPARAM_MIP_Pool_Capacity                       2
CPXPARAM_MIP_Pool_Intensity                      4
2 of 2 MIP starts provided solutions.
MIP start 'm1' defined initial solution with objective -52.1028.
Tried aggregator 1 time.
MIP Presolve eliminated 4 rows and 0 columns.
Reduced MIP has 19 rows, 34 columns, and 106 nonzeros.
Reduced MIP has 30 binaries, 0 generals, 0 SOSs, and 0 indicators.
Presolve time = 0.00 sec. (0.04 ticks)
Probing time = 0.00 sec. (0.01 ticks)
Tried aggregator 1 time.
Reduced MIP has 19 rows, 34 columns, and 106 nonzeros.
Reduced MIP has 30 binaries, 0 generals, 0 SOSs, and 0 indicators.
Presolve time = 0.00 sec. (0.04 ticks)
Probing time = 0.00 sec. (0.01 ticks)
Clique table members: 8.
MIP emphasis: balance optimality and feasibility.
MIP search method: dynamic search.
Parallel mode: deterministic, using up to 1



 ######## Solving Subproblem ######## 


Version identifier: 22.1.1.0 | 2022-11-28 | 9160aff4d
CPXPARAM_Read_DataCheck                          1
CPXPARAM_MIP_Pool_Capacity                       2
CPXPARAM_MIP_Pool_Intensity                      4
2 of 2 MIP starts provided solutions.
MIP start 'm1' defined initial solution with objective -52.1028.
Tried aggregator 1 time.
MIP Presolve eliminated 4 rows and 0 columns.
Reduced MIP has 19 rows, 34 columns, and 106 nonzeros.
Reduced MIP has 30 binaries, 0 generals, 0 SOSs, and 0 indicators.
Presolve time = 0.00 sec. (0.04 ticks)
Probing time = 0.00 sec. (0.01 ticks)
Tried aggregator 1 time.
Reduced MIP has 19 rows, 34 columns, and 106 nonzeros.
Reduced MIP has 30 binaries, 0 generals, 0 SOSs, and 0 indicators.
Presolve time = 0.00 sec. (0.04 ticks)
Probing time = 0.00 sec. (0.01 ticks)
Clique table members: 8.
MIP emphasis: balance optimality and feasibility.
MIP search method: dynamic search.
Parallel mode: deterministic, using up to 1

## Visualization